# Utah Avalanch Data Project

## Introduction

## Starting Data

The initial idea is to draw out each avalanche/conditions observation and log that in a table of observations or catalog. Each observation in the catalog holds whatever data that observer reports (weather, tests, descriptions, avalanche, etc). First we need to try doing this for the first page of observations in this year 2023. Then we match observations to forcasts and see if we can't get some avalanche data. 

In [1]:
import pandas as pd
import os
import requests
import numpy as np
import matplotlib as plt
from bs4 import BeautifulSoup
pd.__version__ #need version>=1.5

'1.5.2'

In [2]:
import sys
print(sys.executable)
print(sys.version)
print(sys.version_info)

C:\Users\etran\Anaconda3\python.exe
3.9.16 (main, Jan 11 2023, 16:16:36) [MSC v.1916 64 bit (AMD64)]
sys.version_info(major=3, minor=9, micro=16, releaselevel='final', serial=0)


I got some good infor on converting html files into pandas dataframes from this article, https://towardsdatascience.com/how-to-get-tables-from-websites-into-pandas-dataframes-e82bd8a0ac59. `

In [3]:
url = 'http://utahavalanchecenter.org/observations'
page = requests.get(url)
soup = BeautifulSoup(page.content, 'html.parser')
tbl = soup.find("table")
#print(tbl.dtype)
page_obs = pd.read_html(str(tbl),extract_links ='all')[0] #extract link so we can take a closer look at the observation
page_obs.head()

,"(Date, None)","(Region, None)","(, None)","(Observer, None)"
0,"(2/9/2023, None)","(Salt Lake, None)","(Observation: White Pine, /observation/75011)","(Gagne, None)"
1,"(2/9/2023, None)","(Uintas, None)","(Observation: Mill Hollow, /observation/75013)","(Gordon, Manship, Collett, None)"
2,"(2/9/2023, None)","(Salt Lake, None)","(Observation: Toots to Boot, /observation/75012)","(Meisenheimer / Staples, None)"
3,"(2/9/2023, None)","(Provo, None)","(Observation: North Fork, /observation/75000)","(Josh Martineau, None)"
4,"(2/9/2023, None)","(Uintas, None)","(Avalanche: Tower Mountain, /avalanche/75010)","(Gordon, Manship, Collett, None)"


In [5]:
extension = page_obs.iloc[0,2][1]
url = 'http://utahavalanchecenter.org' + extension
print(url)
page = requests.get(url)
soup = BeautifulSoup(page.content, 'html.parser')
#print(soup)
fields = []
for section in soup.find_all(attrs={'class':"field-label text_02 bold mb0"}):
    fields.append(section.string)
data = []
for section in soup.find_all(attrs={'class':"text_02 mb2"}):
    data.append(section.string)
print('The fields are', fields)
print(data)

http://utahavalanchecenter.org/observation/75011
The fields are ['Observation Date', 'Observer Name', 'Region', 'Location Name or Route', 'Sky', 'Weather Comments', 'New Snow Depth', 'New Snow Density', 'Snow Surface Conditions', 'Snow Characteristics Comments', 'Red Flags', 'Problem', 'Trend', 'Problem #1 Comments', 'Comments', "Today's Observed Danger Rating", 'Tomorrows Estimated Danger Rating', 'Coordinates']
['2/9/2023', 'Gagne', 'Salt Lake » Little Cottonwood Canyon » White Pine', 'White Pine', 'Clear', 'Calm winds.', '2"', 'Low', 'Powder', 'Wind Crust', "Wind effect was most evident in exposed alpine terrain above about 9,500'. ", 'Recent Avalanches', 'Wind Loading', 'Wind Drifted Snow', 'Decreasing Danger', 'The strong winds on Wednesday created pockets of wind-drifted snow on all aspects at the upper elevations. The drifts were generally only 5-15 cms thick, but some were up to 30 cms thick.', None, None, None, None, None, None, None, 'Moderate', 'Moderate', None]


One problem with this data is that there are some categories such as Red Flags that have multiple entries. Looking at the observation report page (https://utahavalanchecenter.org/node/add/observations) there are up to 7 possible red flags and spaces to name up to two avalanche problems.